In [11]:
import importlib
import datetime
import numpy as np
import pandas as pd
from numpy import random as npr
from modules import deepfm as custom_deepfm, utils
importlib.reload(utils)
importlib.reload(custom_deepfm)

<module 'modules.deepfm' from '/root/ws/SynEvaRec/modules/modules/deepfm.py'>

In [126]:
SEED = 2021
MOVIELENS_DATASET_PATH = "data/data.csv"
REAL_DATASET_SAMPLE_SIZE = 1
CACHE_DIR = "./cache"
SYNTHETIC_USERS_COUNT = 1000
SYNTHETIC_ITEMS_COUNT = 1000
SAMPLE_SIZES = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0]
ALPHA_SAMPLE_RATE = 10
CTGAN_EPOCHS = 10
PART_RATIO = 0.2
N_CHUNKS = 10 

npr.seed(SEED)

In [127]:
real_dataset = pd.read_csv(MOVIELENS_DATASET_PATH)
npr.seed(SEED)
real_dataset = real_dataset.sample(frac=REAL_DATASET_SAMPLE_SIZE)
real_dataset["user_id"] = real_dataset["user_id"].astype('category').cat.codes
real_dataset["item_id"] = real_dataset["movie_id"].astype('category').cat.codes
real_dataset["rating"] = real_dataset["rating"]
real_dataset["timestamp"] = pd.to_datetime(real_dataset["timestamp"], unit="s")
real_dataset = real_dataset.sort_values("timestamp")
real_dataset = real_dataset.drop(["timestamp", "item", "movie_name", "zip"], axis=1)
real_dataset

/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,user_id,movie_id,rating,tag0,tag1,tag2,tag3,tag4,tag5,tag6,...,tag12,tag13,tag14,tag15,tag16,tag17,gender,age,occupation,item_id
456790,6039,858,4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,M,25,6,802
456732,6039,2384,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,M,25,6,2191
456672,6039,593,5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,M,25,6,579
456641,6039,1961,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,M,25,6,1781
456842,6039,2019,5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,M,25,6,1839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373230,4957,2399,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,M,18,7,2206
373293,4957,1407,5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,M,18,7,1305
373229,4957,3264,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,M,18,7,3040
373382,4957,2634,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,M,18,7,2432


In [128]:
def to_category(df, columns):
    for c in columns:
        df[c] = df[c].astype("category")
    return df


In [129]:
model_features = [
    "gender",
    "occupation",
    "age",
    *[f"tag{i}" for i in range(18)]
]

## Generate synthetic data

In [148]:
from sdv.tabular import CTGAN, GaussianCopula, CopulaGAN
from sdv import constraints

occupation_constr = constraints.Positive(
    high="occupation",
    strict=False,
    handling_strategy='reject_sampling'
)

def fit_syn_generator(df):
    model = CTGAN(verbose=True, epochs=CTGAN_EPOCHS, constraints=[occupation_constr])
    df = df.astype("int64", errors="ignore") # Convert all numbers to int64
    model.fit(df.copy())
    return model


def fit_synthetic_generators(real_dataset):
    users = real_dataset[["user_id", "age", "occupation", "gender"]].drop_duplicates().drop("user_id", axis=1)
    items = real_dataset[["movie_id", *[f"tag{n}" for n in range(18)]]].drop_duplicates().drop("movie_id", axis=1)
    
    print(f"Fitting users generator to {users.shape} dataset")
    user_gen = fit_syn_generator(users)
    
    print(f"Fitting items generator to {items.shape} dataset")
    item_gen = fit_syn_generator(items)
    return user_gen, item_gen


def generate_synthetic_data(users_generator, items_generator, n_users=100, n_items=100):
    syn_users = users_generator.sample(n_users)
    syn_items = items_generator.sample(n_items)
    syn_users["user_id"] = range(len(syn_users))
    syn_items["item_id"] = range(len(syn_items))
    syn_users["_merge_key"] = syn_items["_merge_key"] = 1
    
    syn_dataset = pd.merge(syn_items, syn_users, on="_merge_key")
    syn_dataset = syn_dataset.drop(["_merge_key"], axis=1)
    return syn_dataset

AttributeError: module 'sdv.constraints' has no attribute 'Positive'

In [138]:
%%time
real_dataset["occupation"] = real_dataset["occupation"].astype("category")
npr.seed(SEED)
user_gen, item_gen = fit_synthetic_generators(real_dataset)

Fitting users generator to (6040, 3) dataset


/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (7) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/root/.cache/pypoet

Epoch 1, Loss G:  0.7245,Loss D: -0.0220
Epoch 2, Loss G:  0.4891,Loss D:  0.1294
Epoch 3, Loss G:  0.4552,Loss D:  0.0163
Epoch 4, Loss G:  0.4127,Loss D:  0.2238
Epoch 5, Loss G:  0.4286,Loss D: -0.0253
Epoch 6, Loss G:  0.5801,Loss D: -0.2218
Epoch 7, Loss G:  0.3184,Loss D: -0.1271
Epoch 8, Loss G:  0.1961,Loss D: -0.1088
Epoch 9, Loss G:  0.0822,Loss D: -0.0055
Epoch 10, Loss G: -0.0703,Loss D:  0.2878
Fitting items generator to (3706, 18) dataset


/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
/root/.cache/pypoetry/virtualenvs/synevarec-TVdshFRi-py3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "
/root/.cache/pypoet

Epoch 1, Loss G: -0.2978,Loss D: -0.4740
Epoch 2, Loss G: -1.0815,Loss D: -1.1451
Epoch 3, Loss G: -2.5028,Loss D: -2.5327
Epoch 4, Loss G: -3.6852,Loss D: -5.8194
Epoch 5, Loss G: -4.7852,Loss D: -8.4577
Epoch 6, Loss G: -5.6119,Loss D: -9.1592
Epoch 7, Loss G: -6.4012,Loss D: -10.8216
Epoch 8, Loss G: -6.7536,Loss D: -11.7491
Epoch 9, Loss G: -7.0459,Loss D: -11.7051
Epoch 10, Loss G: -7.9836,Loss D: -12.3319
CPU times: user 21.8 s, sys: 950 ms, total: 22.7 s
Wall time: 13.3 s


In [139]:
syn_data = generate_synthetic_data(user_gen, item_gen, n_users=SYNTHETIC_USERS_COUNT, n_items=SYNTHETIC_ITEMS_COUNT)
syn_data = syn_data.sample(frac=0.1)
syn_data = syn_data.reset_index()
syn_data

,index,tag0,tag1,tag2,tag3,tag4,tag5,tag6,tag7,tag8,...,tag13,tag14,tag15,tag16,tag17,item_id,age,occupation,gender,user_id
0,149011,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,149,35,0,F,11
1,68892,0,0,1,0,0,0,1,0,0,...,0,1,0,0,0,68,50,5,F,892
2,834579,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,834,35,16,F,579
3,151632,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,151,25,4,F,632
4,662837,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,662,24,5,M,837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,581911,0,1,0,1,0,0,1,0,0,...,0,1,0,1,0,581,34,13,M,911
99996,392310,0,0,0,0,0,1,0,1,0,...,1,1,0,1,0,392,34,11,F,310
99997,926825,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,926,18,4,F,825
99998,45951,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,45,16,21,F,951


## Train Deep FM models

In [90]:
parts = np.array_split(real_dataset, N_CHUNKS)
p1, p2 = parts[2], parts[9]
p1 = to_category(p1, columns=features + ["rating"])
p2 = to_category(p2, columns=features + ["rating"])


In [91]:
from deeptables.models.deepnets import DeepFM
from deeptables.models import deeptable
from sklearn.model_selection import train_test_split
from tensorflow import keras


def train_deepfm(X, y):
    x_train, x_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.2,
        random_state=SEED
    )
    
    conf = deeptable.ModelConfig(
        metrics=['RootMeanSquaredError'], 
        nets=DeepFM, 
        optimizer=keras.optimizers.RMSprop(),
        auto_discrete=True
    )

    dt = deeptable.DeepTable(config=conf)
    model, history = dt.fit(x_train, y_train, epochs=10)
    score = dt.evaluate(x_test, y_test, batch_size=512, verbose=0)
    print(score)
    return dt

In [92]:
deepfm_p1 = train_deepfm(X=p1[features], y=p1["rating"])

5 class detected, inferred as a [multiclass classification] task
Preparing features taken 0.014193534851074219s
Imputation taken 0.7135334014892578s
Categorical encoding taken 2.1005771160125732s
Discretization taken 3.695487976074219e-05s
Injected a callback [EarlyStopping]. monitor:val_rootmeansquarederror, patience:1, mode:min
>>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['all_categorical_vars: (21)']
---------------------------------------------------------
embeddings:
---------------------------------------------------------
input_dims: [4, 23, 9, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
output_dims: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
dropout: 0.3
---------------------------------------------------------
dense: dropout: 0
batch_normalization: False
--------------------------------------------------------

501/501 [==============================] - 3s 7ms/step - loss: 1.5131 - root_mean_squared_error: 0.3902 - val_loss: 1.4817 - val_root_mean_squared_error: 0.3878
Epoch 2/10
500/501 [============================>.] - ETA: 0s - loss: 1.4559 - root_mean_squared_error: 0.3858WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


501/501 [==============================] - 3s 5ms/step - loss: 1.4559 - root_mean_squared_error: 0.3858 - val_loss: 1.4517 - val_root_mean_squared_error: 0.3855
Epoch 3/10
499/501 [============================>.] - ETA: 0s - loss: 1.4475 - root_mean_squared_error: 0.3853WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


501/501 [==============================] - 2s 5ms/step - loss: 1.4473 - root_mean_squared_error: 0.3853 - val_loss: 1.4418 - val_root_mean_squared_error: 0.3847
Epoch 4/10
487/501 [============================>.] - ETA: 0s - loss: 1.4420 - root_mean_squared_error: 0.3849WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


501/501 [==============================] - 2s 4ms/step - loss: 1.4414 - root_mean_squared_error: 0.3848 - val_loss: 1.4353 - val_root_mean_squared_error: 0.3842
Epoch 5/10
490/501 [============================>.] - ETA: 0s - loss: 1.4343 - root_mean_squared_error: 0.3841WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


...

501/501 [==============================] - 2s 3ms/step - loss: 1.4296 - root_mean_squared_error: 0.3836 - val_loss: 1.4221 - val_root_mean_squared_error: 0.3829
Epoch 7/10
485/501 [============================>.] - ETA: 0s - loss: 1.4252 - root_mean_squared_error: 0.3832WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


501/501 [==============================] - 2s 3ms/step - loss: 1.4256 - root_mean_squared_error: 0.3832 - val_loss: 1.4191 - val_root_mean_squared_error: 0.3826
Epoch 8/10
490/501 [============================>.] - ETA: 0s - loss: 1.4237 - root_mean_squared_error: 0.3830WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


501/501 [==============================] - 2s 3ms/step - loss: 1.4235 - root_mean_squared_error: 0.3830 - val_loss: 1.4193 - val_root_mean_squared_error: 0.3827
Epoch 9/10
491/501 [============================>.] - ETA: 0s - loss: 1.4223 - root_mean_squared_error: 0.3829WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


501/501 [==============================] - 2s 3ms/step - loss: 1.4223 - root_mean_squared_error: 0.3829 - val_loss: 1.4177 - val_root_mean_squared_error: 0.3825
Epoch 10/10
484/501 [===========================>..] - ETA: 0s - loss: 1.4197 - root_mean_squared_error: 0.3826WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


501/501 [==============================] - 2s 3ms/step - loss: 1.4201 - root_mean_squared_error: 0.3827 - val_loss: 1.4144 - val_root_mean_squared_error: 0.3821
Model has been saved to:dt_output/dt_20210817 213429_linear_fm_nets_dnn_nets/linear+fm_nets+dnn_nets.h5
{'loss': 1.409231424331665, 'root_mean_squared_error': 0.38175100088119507}


In [102]:
syn_data = to_category(syn_data, features)
syn_data

,index,tag0,tag1,tag2,tag3,tag4,tag5,tag6,tag7,tag8,...,tag13,tag14,tag15,tag16,tag17,item_id,age,occupation,gender,user_id
0,149011,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,149,24,11,F,11
1,68892,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,68,24,20,M,892
2,834579,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,834,24,13,F,579
3,151632,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,151,25,22,M,632
4,662837,0,1,0,1,0,1,0,1,0,...,1,0,0,0,0,662,17,17,M,837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,581911,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,581,35,8,M,911
99996,392310,0,0,0,1,0,1,0,0,1,...,0,0,0,1,0,392,24,11,M,310
99997,926825,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,926,24,18,M,825
99998,45951,1,0,0,0,0,1,1,0,0,...,0,0,0,0,0,45,24,0,M,951


In [109]:
syn_data["user_id"] = syn_data["user_id"].astype("category").cat.codes
syn_data["item_id"] = syn_data["item_id"].astype("category").cat.codes
syn_data

,index,tag0,tag1,tag2,tag3,tag4,tag5,tag6,tag7,tag8,...,tag13,tag14,tag15,tag16,tag17,item_id,age,occupation,gender,user_id
0,149011,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,149,24,11,F,11
1,68892,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,68,24,20,M,892
2,834579,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,834,24,13,F,579
3,151632,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,151,25,22,M,632
4,662837,0,1,0,1,0,1,0,1,0,...,1,0,0,0,0,662,17,17,M,837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,581911,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,581,35,8,M,911
99996,392310,0,0,0,1,0,1,0,0,1,...,0,0,0,1,0,392,24,11,M,310
99997,926825,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,926,24,18,M,825
99998,45951,1,0,0,0,0,1,1,0,0,...,0,0,0,0,0,45,24,0,M,951


In [104]:
res = deepfm_p1.predict(syn_data[features])

In [115]:
p1_syn["user_id"] = syn_data["user_id"]
p1_syn["item_id"] = syn_data["item_id"]
p1_syn["rating"] = res
p1_syn = p1_syn[["user_id", "item_id", "rating"]]
p1_syn

,user_id,item_id,rating
0,11,149,3
1,892,68,3
2,579,834,3
3,632,151,3
4,837,662,3
...,...,...,...
99995,911,581,1
99996,310,392,3
99997,825,926,3
99998,951,45,3


In [116]:
p1_syn.dtypes

user_id    int16
item_id    int16
rating     int64
dtype: object

In [117]:
def evaluate(df):
    errors = {
        "svd": utils.evaluate_svd(df.copy(), rating_scale=(0, 5)),
        "knn": utils.evaluate_knn(df.copy(), rating_scale=(0, 5)),
        "autorec": utils.evaluate_autorec(df.copy())
    }
    return errors

In [118]:
evaluate(p1_syn)

RMSE: 0.0657
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.3426
Load data finished. Number of users: 1000 Number of items: 1000
IAutoRec.
Network built
Epoch: 0000; RMSE:2.569587968786568; MAE:2.519111428425461
Epoch: 0003; RMSE:0.37815719224640737; MAE:0.2853570456147194
Epoch: 0006; RMSE:1.1238915018926317; MAE:1.1128924575805663
Epoch: 0009; RMSE:0.25467487319899657; MAE:0.18581955291628838
Epoch: 0012; RMSE:0.629451571272938; MAE:0.6085819110035896
Epoch: 0015; RMSE:0.17574672042444933; MAE:0.1540583914756775
Epoch: 0018; RMSE:0.323031455345211; MAE:0.2982868744194508
Epoch: 0021; RMSE:0.1045268206825593; MAE:0.08868959556818008
Epoch: 0024; RMSE:0.1960765944341773; MAE:0.17949238625764846
Epoch: 0027; RMSE:0.08269767870397157; MAE:0.06644620250463486
Epoch: 0030; RMSE:0.10737638734452123; MAE:0.09228669430613518
Epoch: 0033; RMSE:0.0628733824224396; MAE:0.04840213908553123
Epoch: 0036; RMSE:0.08541143965633421; MAE:0.0686866472363472
Epoch: 0039;

Epoch: 0378; RMSE:0.01731149854348397; MAE:0.013583913785219193
Epoch: 0381; RMSE:0.016937775608212746; MAE:0.01330649271607399
Epoch: 0384; RMSE:0.016346981846176115; MAE:0.012852455991506577
Epoch: 0387; RMSE:0.017164998054667417; MAE:0.013440625101327897
Epoch: 0390; RMSE:0.016137777632715185; MAE:0.012693890172243118
Epoch: 0393; RMSE:0.01725403064553978; MAE:0.013547745883464813
Epoch: 0396; RMSE:0.016403645756304406; MAE:0.012841143381595612
Epoch: 0399; RMSE:0.0164086516924471; MAE:0.012908468919992446
Epoch: 0402; RMSE:0.017365206249199003; MAE:0.0136568017244339
Epoch: 0405; RMSE:0.016496474471313007; MAE:0.013000544792413712
Epoch: 0408; RMSE:0.017217113539143687; MAE:0.01352740285396576
Epoch: 0411; RMSE:0.016007933721571995; MAE:0.012615404498577117
Epoch: 0414; RMSE:0.016367233962458854; MAE:0.012870308339595795
Epoch: 0417; RMSE:0.016276479499979457; MAE:0.012787144392728806
Epoch: 0420; RMSE:0.016157193589335456; MAE:0.01274336307644844
Epoch: 0423; RMSE:0.01617317346546

{'svd': 0.06567633749869001,
 'knn': 0.3426042238571631,
 'autorec': 0.014916403176308405}

## Train second deep fm

In [119]:
deepfm_p2 = train_deepfm(X=p2[features], y=p2["rating"])

5 class detected, inferred as a [multiclass classification] task
Preparing features taken 0.015745878219604492s
Imputation taken 0.7067008018493652s
Categorical encoding taken 2.506718635559082s
Discretization taken 3.4809112548828125e-05s
Injected a callback [EarlyStopping]. monitor:val_rootmeansquarederror, patience:1, mode:min
>>>>>>>>>>>>>>>>>>>>>> Model Desc <<<<<<<<<<<<<<<<<<<<<<< 
---------------------------------------------------------
inputs:
---------------------------------------------------------
['all_categorical_vars: (21)']
---------------------------------------------------------
embeddings:
---------------------------------------------------------
input_dims: [4, 22, 9, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
output_dims: [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
dropout: 0.3
---------------------------------------------------------
dense: dropout: 0
batch_normalization: False
--------------------------------------------------------

501/501 [==============================] - 3s 6ms/step - loss: 1.4650 - root_mean_squared_error: 0.3860 - val_loss: 1.4838 - val_root_mean_squared_error: 0.3882
Epoch 2/10
492/501 [============================>.] - ETA: 0s - loss: 1.4462 - root_mean_squared_error: 0.3846WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


501/501 [==============================] - 2s 5ms/step - loss: 1.4461 - root_mean_squared_error: 0.3846 - val_loss: 1.4450 - val_root_mean_squared_error: 0.3847
Epoch 3/10
497/501 [============================>.] - ETA: 0s - loss: 1.4436 - root_mean_squared_error: 0.3844WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


501/501 [==============================] - 2s 5ms/step - loss: 1.4435 - root_mean_squared_error: 0.3844 - val_loss: 1.4422 - val_root_mean_squared_error: 0.3845
Epoch 4/10
495/501 [============================>.] - ETA: 0s - loss: 1.4421 - root_mean_squared_error: 0.3844WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


501/501 [==============================] - 2s 5ms/step - loss: 1.4421 - root_mean_squared_error: 0.3844 - val_loss: 1.4466 - val_root_mean_squared_error: 0.3848
Epoch 5/10
500/501 [============================>.] - ETA: 0s - loss: 1.4396 - root_mean_squared_error: 0.3842WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


...

501/501 [==============================] - 2s 3ms/step - loss: 1.4358 - root_mean_squared_error: 0.3840 - val_loss: 1.4360 - val_root_mean_squared_error: 0.3840
Epoch 7/10
500/501 [============================>.] - ETA: 0s - loss: 1.4325 - root_mean_squared_error: 0.3837WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


501/501 [==============================] - 2s 3ms/step - loss: 1.4326 - root_mean_squared_error: 0.3837 - val_loss: 1.4328 - val_root_mean_squared_error: 0.3838
Epoch 8/10
493/501 [============================>.] - ETA: 0s - loss: 1.4284 - root_mean_squared_error: 0.3833WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


501/501 [==============================] - 2s 3ms/step - loss: 1.4288 - root_mean_squared_error: 0.3833 - val_loss: 1.4249 - val_root_mean_squared_error: 0.3830
Epoch 9/10
486/501 [============================>.] - ETA: 0s - loss: 1.4240 - root_mean_squared_error: 0.3829WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


501/501 [==============================] - 2s 3ms/step - loss: 1.4235 - root_mean_squared_error: 0.3828 - val_loss: 1.4199 - val_root_mean_squared_error: 0.3826
Epoch 10/10
484/501 [===========================>..] - ETA: 0s - loss: 1.4206 - root_mean_squared_error: 0.3825WARNING:tensorflow:Early stopping conditioned on metric `val_rootmeansquarederror` which is not available. Available metrics are: loss,root_mean_squared_error,val_loss,val_root_mean_squared_error


501/501 [==============================] - 2s 3ms/step - loss: 1.4202 - root_mean_squared_error: 0.3825 - val_loss: 1.4151 - val_root_mean_squared_error: 0.3821
Model has been saved to:dt_output/dt_20210817 215742_linear_fm_nets_dnn_nets/linear+fm_nets+dnn_nets.h5
{'loss': 1.4148728847503662, 'root_mean_squared_error': 0.38187068700790405}


In [120]:
res2 = deepfm_p2.predict(syn_data[features])

In [122]:
p2_syn = pd.DataFrame()
p2_syn["user_id"] = syn_data["user_id"]
p2_syn["item_id"] = syn_data["item_id"]
p2_syn["rating"] = res2
p2_syn = p2_syn[["user_id", "item_id", "rating"]]
p2_syn

,user_id,item_id,rating
0,11,149,4
1,892,68,4
2,579,834,4
3,632,151,4
4,837,662,4
...,...,...,...
99995,911,581,4
99996,310,392,4
99997,825,926,4
99998,951,45,4


In [123]:
evaluate(p2_syn)

RMSE: 0.0631
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
Load data finished. Number of users: 1000 Number of items: 1000
IAutoRec.
Network built
Epoch: 0000; RMSE:3.429901735343104; MAE:3.4276559972688556
Epoch: 0003; RMSE:0.5759240629438634; MAE:0.5417439752340317
Epoch: 0006; RMSE:1.3620619559462448; MAE:1.359747031545639
Epoch: 0009; RMSE:0.7855794958673654; MAE:0.7737558711051941
Epoch: 0012; RMSE:0.38686533834662584; MAE:0.36862433149814605
Epoch: 0015; RMSE:0.6016802045359556; MAE:0.5999784693956375
Epoch: 0018; RMSE:0.11459088971981074; MAE:0.09215437119007111
Epoch: 0021; RMSE:0.30480197377997664; MAE:0.3006166349411011
Epoch: 0024; RMSE:0.12890961322888397; MAE:0.11497238183021545
Epoch: 0027; RMSE:0.15178507637940042; MAE:0.1433521115064621
Epoch: 0030; RMSE:0.10413420350457599; MAE:0.09420256247520446
Epoch: 0033; RMSE:0.07371439638835676; MAE:0.06444990894794464
Epoch: 0036; RMSE:0.061503664102606985; MAE:0.05359458267688751
Epoch: 

Epoch: 0378; RMSE:0.019341924203066106; MAE:0.01511210949420929
Epoch: 0381; RMSE:0.019612637867529932; MAE:0.015145809817314148
Epoch: 0384; RMSE:0.01948148210909998; MAE:0.015082020998001098
Epoch: 0387; RMSE:0.019511641896041185; MAE:0.015117180681228637
Epoch: 0390; RMSE:0.019779132889925923; MAE:0.015261293148994446
Epoch: 0393; RMSE:0.01920043340669475; MAE:0.015075903415679931
Epoch: 0396; RMSE:0.01931995302048861; MAE:0.014846287631988525
Epoch: 0399; RMSE:0.01920556420884999; MAE:0.014757409596443177
Epoch: 0402; RMSE:0.018838625864342158; MAE:0.014582132744789124
Epoch: 0405; RMSE:0.01897108928792763; MAE:0.014930115270614623
Epoch: 0408; RMSE:0.018949143283223845; MAE:0.014562688851356507
Epoch: 0411; RMSE:0.018540994678827204; MAE:0.014373075580596924
Epoch: 0414; RMSE:0.018371321885214016; MAE:0.01414532449245453
Epoch: 0417; RMSE:0.018486082791103173; MAE:0.01445866551399231
Epoch: 0420; RMSE:0.018329204924002394; MAE:0.014154404211044311
Epoch: 0423; RMSE:0.0179582212406

{'svd': 0.063102446414593, 'knn': 0.0, 'autorec': 0.016731250333201005}